In [3]:
import json
import os
import pandas as pd
import numpy as np
import scipy.stats
from scipy import stats, ndimage
from scipy.signal import butter, lfilter, medfilt

#Change these for the data

trialLength = 3300
baselineLength = 1000

In [4]:
def calculate_column_wise_average(df):
    columns_to_average = df.columns[5:3305].values
    return df[columns_to_average].mean(axis=0)

In [5]:
all_participants_data = pd.DataFrame()
all_ext_participants_data = pd.DataFrame()

def process_participant(participantID):
    global all_participants_data
    global all_ext_participants_data
    path = "/Users/dhewitt/Data/pps/"
    
    PKL_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID  + "_dataframelist.pkl")
    CSV_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID  + "_Extractednew.csv")
    OUT_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID  + "_PupilDiameterProcessed.csv")
    AV_OUT_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID + "_PupilDiameterProcessed_Av.csv")
    EXT_AV_OUT_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID + "_PupilDiameterProcessed_AvExt.csv")

    data2load = pd.read_csv(CSV_FILE_NAME) 
    dataframe_list = pd.read_pickle(PKL_FILE_NAME)

    #Normalising the timestamp from the first timepoint.
    #Just getting the 'in trial' bit from trial start/cue appearing to the pain stimulus onset.

    Right_extracted_pupil_sizes = list(map(lambda df: np.interp(np.arange(baselineLength+trialLength), df['timestamp'] - df['timestamp'][0], 
                    df['right_pupil_diameter_mm'].to_numpy()), dataframe_list))

    Left_extracted_pupil_sizes = list(map(lambda df: np.interp(np.arange(baselineLength+trialLength), df['timestamp'] - df['timestamp'][0], 
                    df['left_pupil_diameter_mm'].to_numpy()), dataframe_list))

    # Now getting the blinks and interpolating them - starting with the right pupil
    df1 = pd.DataFrame(Right_extracted_pupil_sizes)
    total_mean = df1.stack().mean()
    total_std = df1.stack().std()

    # Get blinks
    data_columns = df1.columns
    blink_indices_list = []
    
    def calculate_blink_indices(row):
        trial_data = row[data_columns]     # Extract the data points for the current trial
        trial_mean = trial_data.mean()
        trial_std = trial_data.std()
        blink_data_points = np.abs(trial_data - trial_mean) > 2 * trial_std     # Identify data points more than 2 standard deviations from the mean
        #blink_data_points = np.abs(trial_data - total_mean) > 3 * total_std
        column_indices = np.where(blink_data_points)[0] # Get the column indices where blink_data_points are True
        blink_indices = [{"Row Index": row.name, "Column Index": data_columns[col_idx]} for col_idx in column_indices] # Create a list of dictionaries with row and column indices
        blink_indices_list.extend(blink_indices)
    
    df1.apply(calculate_blink_indices, axis=1)
    blink_indices_df = pd.DataFrame(blink_indices_list)

    sorted_df = blink_indices_df.sort_values(by=['Row Index', 'Column Index'])
    diff = sorted_df['Column Index'].diff()
    # Identify gaps in column indices and create a new group when gaps are not equal to 1
    group = (diff != 1).cumsum()
    # Group the data by 'Row Index' and the calculated 'group'
    grouped_data = sorted_df.groupby(['Row Index', group], as_index=False)['Column Index'].agg(['first', 'last'])

    # Rename the columns for clarity
    grouped_data.columns = ['Start Column Index', 'End Column Index']
    grouped_data['Total Dur'] = grouped_data['End Column Index']-grouped_data['Start Column Index']

    threshold = 150;
    grouped_data = grouped_data[grouped_data['Total Dur']>=threshold].reset_index().drop(columns='Column Index')

    interpolated_df1 = df1.copy()
    # Iterate over the rows and matching columns in filtered_grouped_data
    for _, row_data in grouped_data.iterrows():
        row_index = row_data['Row Index']
        start_col_index = row_data['Start Column Index']
        end_col_index = row_data['End Column Index']

        # Ensure the row index is present in the DataFrame
        if row_index in interpolated_df1.index:
            # Replace matching data with NaN for interpolation
            print(row_index, start_col_index, end_col_index)
            interpolated_df1.iloc[row_index, start_col_index:end_col_index + 1] = np.nan

    # Interpolate the missing values using linear interpolation
    interpolated_df1 = interpolated_df1.interpolate(method='linear', axis=1, limit_direction='both')

    #matching_df.to_csv('/Users/dhewitt/Data/pps/P03/P03_matching.csv')
    #interpolated_df1.to_csv('/Users/dhewitt/Data/pps/P03/P03_interpdf_nan00.csv')

    ###Getting rid of any trials with too many interpolations - 25% of all trials

    # Filter out 'Row Index' in grouped_data with 'Total Dur' greater than 825
    rows_to_remove = grouped_data[grouped_data['Total Dur'] > 1025]['Row Index']

    # Drop rows in interpolated_df2 corresponding to the selected 'Row Index'
    interpolated_df1 = interpolated_df1.drop(rows_to_remove, errors='ignore')
    interpolated_df1.reset_index(drop=True, inplace=True)

    # Now the left pupil

    df2 = pd.DataFrame(Left_extracted_pupil_sizes)
    total_mean = df2.stack().mean()
    total_std = df2.stack().std()

    # Create an empty list to store dictionaries of blink indices
    data_columns = df2.columns
    blink_indices_list = []
    
    def calculate_blink_indices(row):
        trial_data = row[data_columns]     # Extract the data points for the current trial
        trial_mean = trial_data.mean()
        trial_std = trial_data.std()
        blink_data_points = np.abs(trial_data - trial_mean) > 2 * trial_std     # Identify data points more than 2 standard deviations from the mean
        #blink_data_points = np.abs(trial_data - total_mean) > 3 * total_std
        column_indices = np.where(blink_data_points)[0] # Get the column indices where blink_data_points are True
        blink_indices = [{"Row Index": row.name, "Column Index": data_columns[col_idx]} for col_idx in column_indices] # Create a list of dictionaries with row and column indices
        blink_indices_list.extend(blink_indices)
    
    
    df2.apply(calculate_blink_indices, axis=1)
    blink_indices_df = pd.DataFrame(blink_indices_list)

    sorted_df = blink_indices_df.sort_values(by=['Row Index', 'Column Index'])
    diff = sorted_df['Column Index'].diff()
    # Identify gaps in column indices and create a new group when gaps are not equal to 1
    group = (diff != 1).cumsum()
    # Group the data by 'Row Index' and the calculated 'group'
    grouped_data = sorted_df.groupby(['Row Index', group], as_index=False)['Column Index'].agg(['first', 'last'])
    grouped_data.columns = ['Start Column Index', 'End Column Index']
    grouped_data['Total Dur'] = grouped_data['End Column Index']-grouped_data['Start Column Index']

    threshold = 150;

    grouped_data = grouped_data[grouped_data['Total Dur']>=threshold].reset_index().drop(columns='Column Index')

    interpolated_df2 = df2.copy()
    # Iterate over the rows and matching columns in grouped_data
    for _, row_data in grouped_data.iterrows():
        row_index = row_data['Row Index']
        start_col_index = row_data['Start Column Index']
        end_col_index = row_data['End Column Index']

        # Ensure the row index is present in the DataFrame
        if row_index in interpolated_df2.index:
            print(row_index, start_col_index, end_col_index)
            # Replace matching data with NaN for interpolation
            interpolated_df2.iloc[row_index, start_col_index:end_col_index + 1] = np.nan

    # Interpolate the missing values using linear interpolation
    interpolated_df2 = interpolated_df2.interpolate(method='linear', axis=1, limit_direction='both')

    #df2.to_csv('/Users/dhewitt/Data/pps/P03/P03_matching.csv')
    #interpolated_df2.to_csv('/Users/dhewitt/Data/pps/P03/P03_interpdf2_monday.csv')

    ###Getting rid of any trials with too many interpolations - 25% of all trials

    # Filter out 'Row Index' in grouped_data with 'Total Dur' greater than 825
    rows_to_remove = grouped_data[grouped_data['Total Dur'] > 825]['Row Index']
    interpolated_df2 = interpolated_df2.drop(rows_to_remove, errors='ignore')
    interpolated_df2.reset_index(drop=True, inplace=True)

    ##Filtering

    ## Apply the filter to interpolated_df1 and interpolated_df2
    #filtered_interpolated_df1 = medfilt(interpolated_df1, 3)
    #filtered_interpolated_df2 =  medfilt(interpolated_df2, 3)
    filtered_interpolated_df1 = interpolated_df1
    filtered_interpolated_df2 =  interpolated_df2

    #filtered_interpolated_df1 = ndimage.median_filter(interpolated_df1, size=11)
    #filtered_interpolated_df2 =  ndimage.median_filter(interpolated_df2, size=11)

    ## Create DataFrames from the filtered arrays
    interpolated_df1 = pd.DataFrame(filtered_interpolated_df1, columns=interpolated_df1.columns)
    interpolated_df2 = pd.DataFrame(filtered_interpolated_df2, columns=interpolated_df2.columns)

    # Baseline correcting
    baselineStart=250
    baselineEnd=750

    baselineAv = interpolated_df1.iloc[:,baselineStart:baselineEnd].mean(axis=1)
    interpolated_df1 = interpolated_df1.apply(lambda x: x - baselineAv)

    baselineAv = interpolated_df2.iloc[:,baselineStart:baselineEnd].mean(axis=1)
    interpolated_df2 = interpolated_df2.apply(lambda x: x - baselineAv)

    # Concatenating left and right eyes

    df1 = pd.DataFrame(interpolated_df1) #right pupil sizes after blink interpolation
    df2 = pd.DataFrame(interpolated_df2) #left pupil sizes after blink interpolation

    botheyesdf = (df1+df2)/2

    botheyesdf.drop(botheyesdf.iloc[:,0:baselineLength], inplace=True, axis=1) #getting rid of the baseline
    columns = dict(map(reversed, enumerate(botheyesdf.columns))) #resetting the index
    botheyesdf = botheyesdf.rename(columns=columns)

    df = pd.DataFrame(data2load)
    newdf = pd.concat([df,botheyesdf],axis=1)

    # Creating averages for each column (i.e., over all trials for each condition)

    condMiddle = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & ((newdf['condorder'] == 1) | (newdf['condorder'] == 3))]
    Middle_cond_average = calculate_column_wise_average(condMiddle)

    condLeft = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & ((newdf['condorder'] == 1) | (newdf['condorder'] == 3))]
    Left_cond_average = calculate_column_wise_average(condLeft)

    condRight = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & ((newdf['condorder'] == 1) | (newdf['condorder'] == 3))]
    Right_cond_average = calculate_column_wise_average(condRight) 

    extMiddle = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & ((newdf['condorder'] == 2) | (newdf['condorder'] == 4))]
    Middle_ext_average = calculate_column_wise_average(extMiddle)

    extLeft = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & ((newdf['condorder'] == 2) | (newdf['condorder'] == 4))]
    Left_ext_average = calculate_column_wise_average(extLeft)

    extRight = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & ((newdf['condorder'] == 2) | (newdf['condorder'] == 4))]
    Right_ext_average = calculate_column_wise_average(extRight)    

    # Saving single subject data for pain-cue-block comparisons

    frames = [Middle_cond_average, Left_cond_average, Right_cond_average, Middle_ext_average, Left_ext_average, Right_ext_average]
    avscore = pd.concat(frames, axis=1, keys = ["CM", "CL", "CR", "EM", "EL", "ER"])
    avscore.to_csv(AV_OUT_FILE_NAME)
    
    # Saving single subject data for extinction-tonic pain comparisons
    
    extMiddle1 = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & (newdf['condorder'] == 2)]
    Middle_ext_average1 = calculate_column_wise_average(extMiddle1)

    extLeft1 = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & (newdf['condorder'] == 2)]
    Left_ext_average1 = calculate_column_wise_average(extLeft1)

    extRight1 = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & (newdf['condorder'] == 2)]
    Right_ext_average1 = calculate_column_wise_average(extRight1)    
    
    extMiddle2 = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & (newdf['condorder'] == 4)]
    Middle_ext_average2 = calculate_column_wise_average(extMiddle2)

    extLeft2 = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & (newdf['condorder'] == 4)]
    Left_ext_average2 = calculate_column_wise_average(extLeft2)

    extRight2 = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & (newdf['condorder'] == 4)]
    Right_ext_average2 = calculate_column_wise_average(extRight2)    
    
    frames = [Middle_ext_average1, Left_ext_average1, Right_ext_average1, Middle_ext_average2, Left_ext_average2, Right_ext_average2]
    avscore2 = pd.concat(frames, axis=1, keys = ["EM1", "EL1", "ER1", "EM2", "EL2", "ER2"])
    avscore2.to_csv(EXT_AV_OUT_FILE_NAME)
    
    # Read the CSV file for the current participant
    participant_data = pd.read_csv(AV_OUT_FILE_NAME)
    participant_data["Participant"] = participantID
    all_participants_data = pd.concat([all_participants_data, participant_data], ignore_index=True)
    
    ext_participant_data = pd.read_csv(EXT_AV_OUT_FILE_NAME)
    ext_participant_data["Participant"] = participantID
    all_ext_participants_data = pd.concat([all_ext_participants_data, ext_participant_data], ignore_index=True)    
    

In [6]:
# List of participant IDs
#participant_ids = ["P02", "P03", "P04", "P05", "P06", "P08", "P09", "P10", "P11", "P12", "P13", "P14", "P15", "P16", "P17", "P19", "P20", "P21", "P22", "P24", "P25", "P26", "P27", "P28", "P29","P30"]  # Add more participant IDs as needed
participant_ids = ["02", "03", "04", "05", "06", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "19", "20", "21", "22", "24", "25", "26", "27", "28", "29","30"] 

# Loop over each participant
for participantID in participant_ids:
    # ... (The existing code for processing a single participant will go here)
    process_participant(participantID)
    
# Concatenate all participant data into a single DataFrame
all_participants_data.to_csv("/Users/dhewitt/Data/pps/Exports/Combined_PupilDiameter.csv", index=False)
all_ext_participants_data.to_csv("/Users/dhewitt/Data/pps/Exports/Combined_PupilDiameterExt.csv", index=False)

0 2048 2255
6 1829 2165
7 4104 4269
14 1784 2065
21 3984 4299
24 1167 1412
26 4140 4299
27 4114 4299
32 2689 3045
33 4071 4299
36 4114 4299
37 93 354
39 665 853
41 1118 1382
42 0 242
42 350 592
44 3876 4094
53 86 264
55 219 381
56 3882 4053
57 0 181
58 1539 1689
60 3848 4007
61 886 1095
63 915 1110
65 1167 1388
66 269 459
72 3867 4031
74 1120 1368
81 851 1119
82 104 380
83 2269 2502
84 1740 2012
85 1781 2017
85 4113 4299
87 0 156
94 1860 2314
98 1660 1821
99 1740 1918
101 1103 1295
102 684 871
108 1858 2138
112 3305 3550
112 4050 4299
113 36 332
117 91 369
119 0 182
121 3601 3793
124 2656 2955
130 1248 1475
136 958 1180
146 3204 3567
152 2 164
154 3953 4299
169 2376 2715
170 4065 4299
180 395 661
181 2257 2583
182 3303 3454
185 3283 3607
186 3995 4299
195 1446 1668
198 3451 3644
199 4030 4299
200 0 166
201 1297 1467
203 3962 4299
204 2444 2694
205 2390 2713
206 2660 2994
207 2004 2356
208 1282 1526
210 2853 3172
212 1821 2061
213 1321 1609
217 4131 4299
219 0 173
222 4125 4299
225 0 22

0 2875 3124
3 1995 2174
5 1883 2172
6 3127 3291
8 1109 1356
11 172 370
12 4115 4299
16 430 598
17 1899 2160
18 1891 2241
20 2066 2318
23 0 354
24 2240 2520
27 3417 3629
28 1218 1394
29 298 460
31 0 162
34 0 306
35 4131 4299
37 317 648
38 0 178
44 4073 4299
45 2595 2881
51 3930 4226
52 602 826
54 2107 2469
57 0 207
61 3937 4116
72 48 262
73 92 390
75 2042 2194
76 3673 3901
79 552 802
80 0 159
85 0 173
86 1971 2213
88 2268 2600
89 1606 1985
96 4024 4299
97 1941 2201
98 3714 4031
101 2603 2838
103 21 319
103 4051 4299
107 3956 4154
109 3528 3821
110 2845 2997
119 2608 2814
122 810 1212
126 480 832
127 2909 3184
128 2556 2769
129 3434 3772
133 405 595
136 2430 2657
140 1998 2262
141 101 270
142 191 390
144 2807 3128
148 203 387
151 278 536
158 143 437
159 2189 2438
161 0 257
161 497 708
162 546 894
163 3475 3784
168 430 611
169 1896 2046
170 1985 2225
175 1198 1440
178 4124 4299
182 2049 2324
183 4120 4299
185 2449 2656
187 1523 1711
189 11 362
192 2246 2478
193 0 265
193 2073 2274
194 295

0 1099 1402
2 1861 2110
4 1418 1606
5 3949 4128
6 0 341
12 825 1142
13 480 690
14 4010 4299
19 71 230
22 0 166
27 2983 3172
32 2357 2941
33 1640 1880
37 0 599
44 1613 1919
47 0 161
49 2809 2972
50 3220 3384
53 1997 2302
57 3269 3447
80 3112 3354
90 2819 3119
91 0 425
92 0 259
95 1117 1313
97 2970 3219
100 44 215
102 0 315
106 212 599
108 1694 2295
116 3701 4299
117 0 401
117 498 710
118 3452 3828
120 1479 1911
122 3210 3600
123 716 875
126 870 1473
134 0 477
136 4009 4299
139 3362 3790
139 3898 4060
141 1033 1525
142 2226 2437
143 2209 2644
144 3965 4299
145 3912 4299
158 1033 1461
159 3203 3468
177 2385 2633
181 1971 2277
196 2085 2361
197 228 403
200 2019 2324
201 49 199
202 2903 3059
204 123 376
207 5 156
211 2996 3277
212 3072 3282
216 538 787
220 1587 1775
232 1982 2242
233 1813 2058
234 1083 1236
235 1752 2099
239 3203 3775
243 1242 1400
246 1588 1863
249 1766 1998
256 2022 2254
261 1595 1813
262 1976 2361
273 1574 1933
274 1584 1859
279 1223 1438
281 4008 4299
282 1467 1761
283 

1 996 1169
2 1240 1398
8 539 710
11 2216 2380
12 0 319
13 3954 4182
15 1481 1644
15 4137 4299
16 1710 1894
17 4004 4299
20 1839 2345
35 3942 4152
36 1950 2629
39 679 866
42 568 732
42 1907 2156
43 2948 3104
45 1970 2206
49 3995 4299
51 1609 1826
52 1104 1437
58 244 469
60 1868 2049
61 1906 2095
64 1963 2139
65 1931 2092
65 2445 2818
67 1952 2190
68 68 259
70 1278 1499
71 1647 1847
71 2028 2335
73 0 281
75 258 451
75 1732 1943
76 1843 2070
78 1995 2168
79 1920 2148
80 1584 1854
81 1900 2157
81 3711 3931
82 778 1063
82 1607 1836
85 1983 2257
88 2042 2382
89 696 885
90 3108 3405
91 1938 2152
92 1854 2118
95 0 178
96 1898 2183
98 3239 3469
99 33 388
100 73 379
102 0 164
103 3077 3329
105 2417 2795
106 3929 4224
107 1558 1816
108 2074 2238
109 3418 3572
110 2202 2498
111 1531 1682
113 1961 2156
120 25 293
121 1899 2098
122 3666 3834
123 3069 3285
124 2038 2250
124 4085 4251
125 2096 2395
126 1128 1294
132 1805 1962
133 1524 1796
134 550 734
135 525 721
135 1930 2097
138 2092 2279
142 2068 2

1 2358 2516
2 1848 2223
4 4144 4299
6 4132 4299
8 3011 3364
14 4096 4299
16 1898 2075
16 2903 3094
27 1940 2159
28 418 636
29 1322 1496
29 1522 1707
30 649 815
32 2453 2642
34 784 991
35 3139 3379
36 2291 2480
39 742 1001
40 1334 1682
42 3129 3330
43 3939 4299
44 2389 2725
48 1081 1240
50 324 652
51 2458 2728
54 1055 1341
57 1604 1756
58 0 214
59 2601 2924
60 911 1070
61 3655 4002
61 4145 4299
62 37 212
62 1624 1857
63 1953 2116
69 236 482
70 3576 3957
72 3016 3341
73 1765 1932
74 2069 2380
77 2986 3181
78 2282 2530
80 956 1112
85 1949 2214
86 1904 2075
87 2132 2282
88 3594 3905
88 4124 4299
90 3334 4014
94 118 283
95 81 251
96 1744 1905
97 3417 3645
156 221 396
166 2049 2204
167 2027 2384
179 814 1064
180 1922 2229
181 1911 2323
183 1951 2171
184 1916 2262
185 1529 1693
187 2088 2380
188 1882 2116
190 2283 2498
193 2145 2365
194 3853 4017
195 1927 2137
197 1903 2083
199 3786 3979
200 2080 2240
201 604 978
202 2581 2750
203 2822 3070
204 3220 3389
205 2312 2695
209 2190 2543
211 2058 2

...

1 1889 2123
3 1917 2130
6 2109 2452
7 0 288
11 4063 4299
13 3294 3603
14 2426 2722
16 0 318
20 3766 4299
22 222 389
23 1855 2162
25 1947 2239
26 1885 2194
28 0 178
33 2075 2260
34 2040 2365
35 1983 2289
37 4040 4299
38 244 493
39 4052 4299
41 3992 4299
43 1906 2197
44 1562 1910
59 532 693
61 558 755
62 2841 3082
63 88 393
64 1091 1316
67 1241 1527
71 2129 2348
72 2072 2263
73 141 385
79 2057 2484
81 98 278
84 0 281
85 3503 3905
86 556 772
88 90 377
91 3068 3460
94 1050 1308
95 1940 2307
97 4141 4299
100 3300 3641
101 0 162
103 1860 2164
105 2154 2513
106 3590 3867
109 1897 2103
118 2987 3181
119 1572 1745
128 107 408
131 2079 2280
132 2097 2439
135 3645 3966
137 36 301
140 675 1029
142 1962 2218
143 3932 4107
144 3204 3430
145 439 710
146 1859 2225
148 2170 2477
150 3253 3639
151 3180 3434
153 1424 1674
155 2641 2912
156 2345 2692
159 3457 3695
163 64 351
165 3709 3878
170 211 368
172 3124 3277
174 2928 3289
181 3242 3652
182 2849 3073
187 215 411
190 4148 4299
191 0 162
192 1203 1447


0 1942 2236
1 1818 2169
2 1239 1436
3 3310 3593
6 2202 2369
7 2202 2381
8 1718 1981
10 1931 2169
11 1924 2286
13 1931 2183
15 1694 1930
18 1919 2171
19 2742 2902
20 372 550
20 3388 3618
21 1576 1822
22 1871 2302
24 4011 4299
26 3059 3271
27 293 551
28 879 1157
29 2038 2294
30 2249 2538
34 2485 2664
35 1878 2146
35 3922 4075
36 2031 2308
37 172 416
38 2566 2772
41 1919 2159
42 1917 2286
43 0 183
47 2122 2403
48 3877 4173
49 1897 2156
51 2179 2414
52 2059 2300
53 1678 1889
54 4040 4299
55 1584 1875
57 726 930
58 370 574
60 2063 2322
62 2158 2409
64 687 972
65 1992 2237
66 1880 2270
67 1890 2256
69 2321 2590
71 3326 3497
72 2016 2344
73 1862 2176
75 698 956
77 2001 2385
78 3971 4150
79 1912 2205
80 1741 1895
83 1776 2014
85 0 317
89 1343 1579
91 2217 2397
97 1859 2027
98 1976 2231
100 2142 2296
102 588 898
104 2146 2428
105 1257 1531
108 755 1004
109 1901 2257
112 785 970
113 3045 3290
114 2099 2311
117 1932 2220
118 911 1145
125 0 161
126 693 953
127 3426 3746
129 2907 3099
130 1984 2170

20 719 916
22 2141 2383
24 626 1023
25 870 1127
29 269 547
30 890 1230
39 2045 2218
40 3491 3645
41 4129 4299
49 1839 2076
54 2346 2595
57 4022 4299
61 4118 4299
64 4005 4299
65 1966 2315
66 422 610
75 1851 2087
76 530 841
78 0 267
81 1875 2125
83 1682 1958
84 4086 4299
86 4143 4299
93 1739 1926
95 1860 2092
96 0 223
100 2202 2449
107 610 815
108 2854 3074
112 1955 2138
113 2200 2450
114 1333 1499
117 3120 3291
118 2153 2435
122 333 632
125 136 347
131 3616 3772
135 2708 2982
140 170 356
148 1538 1759
151 0 222
156 1209 1478
157 947 1153
159 1498 1663
167 1795 1998
169 1089 1368
171 1489 1732
176 1669 1891
178 2173 2328
180 1221 1443
183 1772 2019
186 2554 2888
187 225 464
193 0 206
195 2319 2626
197 1725 1936
198 1837 2008
202 1662 1949
205 4110 4299
210 1894 2052
212 1795 2416
213 1489 1724
217 3918 4092
218 1873 2081
220 3971 4157
221 1826 2141
225 1899 2181
226 1767 2135
228 1500 1823
229 2131 2299
230 970 1295
240 4072 4299
241 2212 2448
243 1834 2139
246 1722 2048
247 2558 2710
2

0 2136 2507
2 1035 1233
3 4090 4299
5 2530 2867
11 1944 2185
15 3035 3252
16 770 1022
17 3196 3440
18 3110 3374
19 2184 2397
20 1911 2130
21 2048 2213
22 1039 1203
23 2521 2719
24 3817 4021
25 3993 4299
29 1965 2458
34 1935 2401
35 2158 2501
36 142 409
36 2023 2392
37 0 218
39 487 688
40 96 310
41 1983 2310
46 0 217
47 843 1091
49 451 709
50 1912 2371
53 2882 3179
56 1004 1197
58 1196 1428
59 41 358
59 1124 1383
60 1947 2192
64 1872 2120
65 2748 3071
66 1206 1423
66 3569 3789
67 3076 3336
69 618 859
70 1921 2480
71 1933 2277
72 2681 2968
78 334 599
78 1226 1442
79 1945 2585
82 1953 2200
83 22 216
84 1110 1353
85 2639 2820
86 1918 2514
87 2610 3132
89 884 1120
91 2866 3337
92 1067 1405
97 2651 2954
100 984 1231
102 926 1228
104 2786 2995
106 1955 2287
107 1928 2182
111 2024 2498
112 3444 3653
113 115 272
114 835 1075
115 1868 2089
116 2361 2643
117 2716 2973
118 1543 1745
119 4077 4299
121 2829 3026
123 1059 1261
129 1952 2171
133 2422 2580
136 1744 2001
138 828 1122
140 3397 3734
141 1

0 2937 3100
1 2089 2242
1 2255 2564
2 2071 2255
4 3518 3772
5 1930 2196
7 2076 2462
10 2079 2366
11 727 897
12 1941 2177
13 417 675
14 0 305
16 0 150
18 1968 2297
19 2596 2858
23 0 159
24 863 1126
25 960 1184
29 627 989
33 4135 4299
35 557 711
40 3965 4299
46 1983 2420
48 1928 2172
49 2288 2441
52 2066 2279
54 1887 2221
62 4071 4299
64 1937 2232
65 0 201
67 288 523
69 1878 2168
71 1929 2344
74 4068 4299
76 3984 4299
77 2125 2278
78 436 744
81 548 884
85 0 179
88 1952 2261
89 1890 2234
91 1005 1301
92 1899 2218
94 3906 4201
99 4040 4299
104 1927 2168
105 1804 2148
107 1751 2103
108 0 277
110 1927 2218
111 2008 2303
112 1698 1991
113 1890 2248
115 2169 2335
116 1862 2176
118 900 1201
120 1927 2189
121 1895 2164
123 2060 2393
126 1715 2050
127 2033 2189
128 1898 2199
133 1635 1934
142 2123 2341
149 1752 2229
152 1136 1383
153 989 1325
156 4128 4299
159 1768 2054
160 92 453
161 0 260
163 3145 3440
166 1159 1403
168 199 490
169 285 522
171 149 370
171 1868 2215
178 1869 2214
181 2126 2347
1